# 2 Player FPSB Auction with assymetric uniform valuation distributions

## Imports

In [ ]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
import time
from timeit import default_timer as timer

from functools import partial

import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

from bnelearn.strategy import NeuralNetStrategy, ClosureStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import AuctionEnvironment

from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

# set up matplotlib
is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display
plt.rcParams['figure.figsize'] = [8, 5]
    
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

# Use specific cuda gpu if desired (i.e. for running multiple experiments in parallel)
specific_gpu = 7
if cuda and specific_gpu:
    torch.cuda.set_device(specific_gpu)

print(device)
if cuda: print(torch.cuda.current_device())

## Settings


In [ ]:
# log in notebook folder
# alternative for shared access to experiments:
# log_root = os.path.abspath('/srv/bnelearn-experiments/')
log_root = os.path.abspath('.')
run_comment = ''

## Experiment setup
n_players = 2
n_items = 1
# valuation distribution
# both players should have same lower bound
u_lo =   5.
u1_hi = 15.
u2_hi = 25.
u_his = [u1_hi, u2_hi]

def strat_to_bidder(strategy, batch_size, player_position):
    return Bidder.uniform(u_lo, u_his[player_position], strategy, player_position=player_position, batch_size = batch_size)

## Environment settings
#training batch size
batch_size = 2**16
eval_batch_size = 2**25

# strategy model architecture
input_length = 1
hidden_nodes = [5, 5]
hidden_activations = [nn.SELU(), nn.SELU()]

# optimization params
epoch = 5000
learning_rate = 2e-1
lr_decay = False
lr_decay_every = 1000
lr_decay_factor = 0.5
baseline = True
momentum = 0.6

sigma = .02 #ES noise parameter
n_perturbations = 64

# plot and log training options
plot_epoch = 50
plot_points = min(100, batch_size)

plot_xmin = u_lo
plot_xmax = u2_hi
plot_ymin = 0
plot_ymax = 15

## Setting up the Environment

For $v_1 \sim U(\alpha, \beta_1)$, $v_2 \sim U(\alpha, \beta_2)$, and with

$$c = \frac{1}{(\beta_1 - \alpha)²} -\frac{1}{(\beta_2 - \alpha)²}, $$

the equilibrium bids are given by

$$b_1^*(v_1) = \alpha + \frac{v_1 - \alpha}{1 + \sqrt{1-c(v_1-\alpha)²}} $$

$$b_2^*(v_2) = \alpha + \frac{v_2 - \alpha}{1 + \sqrt{1-c(v_2-\alpha)²}} $$

(See https://link.springer.com/article/10.1007/BF01271133

The expected utility in the bne can then be calculated using

$$E[u_{BNE}] = \int_{0}^{\infty}{P(win | b) * u(b,v | win) *f(v) dv} = ???$$


In [ ]:
# for evaluation
# helper constant
c = 1 / (u1_hi - u_lo)**2 - 1 / (u2_hi - u_lo)**2
def optimal_bid(valuation: torch.Tensor or np.ndarray or float,
                player_position: int) -> torch.Tensor:
    #print(c)
    
    if not isinstance(valuation, torch.Tensor):
        valuation = torch.tensor(valuation, dtype=torch.float)
    #unsqueeze if simple float
    if valuation.dim() == 0:
        valuation.unsqueeze_(0)
    
    if player_position == 0:
        # weak player
        return u_lo + (valuation - u_lo) / (1 + torch.sqrt(1 - c*(valuation - u_lo)**2))
    elif player_position == 1:
        # strong player
        return u_lo + (valuation - u_lo) / (1 + torch.sqrt(1 + c*(valuation - u_lo)**2))


def log_once(writer, e):
    """Everything that should be logged only once on initialization."""
    writer.add_scalar('debug/total_model_parameters', n_parameters, e)
    writer.add_scalar('debug/eval_batch_size', eval_batch_size, e)
    writer.add_text('hyperparams/neural_net_spec', str(model_1), 0)    
    #writer.add_scalar('debug/eval_batch_size', eval_batch_size, e)
    writer.add_graph(model_1, env.agents[0].valuations)    

def log_hyperparams(writer, e):
    writer.add_scalar('hyperparams/batch_size', batch_size, e)
    writer.add_scalar('hyperparams/batch_size', batch_size, e)
    writer.add_scalar('hyperparams/learning_rate', learning_rate, e)
    writer.add_scalar('hyperparams/momentum', momentum, e)
    writer.add_scalar('hyperparams/sigma', sigma, e)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations, e)
    
    
def log_utilities_per_player(global_writer, player_writers, u, u_vs_bne, e):
    """log scalar for each player. Tensor should be of shape n_players"""
    
    epsilons_rel = eps_rel(u_vs_bne)
    epsilons_abs = eps_abs(u_vs_bne)
    
    for i,w in enumerate(player_writers):
        w.add_scalar('eval/utility', u[i], e)
        w.add_scalar('eval/utility_vs_bne', u_vs_bne[i], e)
        w.add_scalar('debug/epsilon_absolute', epsilons_abs[i], e)
        w.add_scalar('eval/epsilon_relative', epsilons_rel[i], e)
    
    global_writer.add_scalar('eval/epsilon_relative', epsilons_rel.mean(),e)
    global_writer.add_scalar('debug/epsilon_absolute', epsilons_abs.mean(),e)

v1_opt = np.linspace(u_lo, u1_hi, 25)
b1_opt = optimal_bid(v1_opt, 0).numpy()
v2_opt = np.linspace(u_lo, u2_hi, 50)
b2_opt = optimal_bid(v2_opt, 1).numpy()
    
def plot_bid_function(fig, v1,b1, v2, b2, writer=None, e=None, plot_points=plot_points):
    #plot_points = min(plot_points, len(v1), len(v2))
    
    # subsample points and plot    
    v1 = v1.detach().cpu().numpy()[:plot_points]
    b1 = b1.detach().cpu().numpy()[:plot_points]
    v2 = v2.detach().cpu().numpy()[:plot_points]
    b2 = b2.detach().cpu().numpy()[:plot_points]
    
    fig = plt.gcf()
    plt.cla()
    plt.xlim(plot_xmin, plot_xmax)
    plt.ylim(plot_ymin, plot_ymax)
    plt.plot(v1,b1, 'bo', v1_opt, b1_opt, 'b--', v2,b2, 'ro', v2_opt,b2_opt, 'r--')
    #if is_ipython:
        #display.clear_output(wait=True)
    display.display(plt.gcf())
    if writer:
        writer.add_figure('eval/bid_function', fig, e)  

Initialize the model.
We'll ensure the initialization provides positive outputs on the domain we are interested in, as otherwise we can't learn.

In [ ]:
# initialize models
model_1 = NeuralNetStrategy(input_length,                            
                            hidden_nodes = hidden_nodes,
                            hidden_activations = hidden_activations,
                            requires_grad=False,
                            ensure_positive_output = torch.tensor([float(u1_hi)])
                            ).to(device)
   

model_2 = NeuralNetStrategy(input_length,
                            hidden_nodes = hidden_nodes,
                            hidden_activations = hidden_activations,
                            requires_grad=False,
                            ensure_positive_output = torch.tensor([float(u2_hi)])
                            ).to(device)

bidder_1 = strat_to_bidder(model_1, batch_size, player_position=0)
bidder_2 = strat_to_bidder(model_2, batch_size, player_position=1)

mechanism = FirstPriceSealedBidAuction(cuda = True)
env = AuctionEnvironment(mechanism,
                  agents = [bidder_1, bidder_2],
                  batch_size = batch_size,
                  n_players =n_players,
                  strategy_to_player_closure = strat_to_bidder
                 )
optimizer_1 = ES(model=model_1, environment = env,
                 lr = learning_rate, momentum=momentum,
                 sigma=sigma, n_perturbations=n_perturbations, baseline=baseline,
                 strat_to_player_kwargs={"player_position":0}
                )
optimizer_2 = ES(model=model_2, environment = env,
                 lr = learning_rate, momentum=momentum,
                 sigma=sigma, n_perturbations=n_perturbations, baseline=baseline,
                 strat_to_player_kwargs={"player_position":1}
                )

print(model_1)
n_parameters = sum([p.numel() for p in model_1.parameters()])
print('Total parameters: ' + str(n_parameters))

## Set up equilibrium-environment

In [ ]:
bne_strategies = [
    ClosureStrategy(partial(optimal_bid, player_position=i))
    for i in range(n_players)
]

bne_env = AuctionEnvironment(
    mechanism,
    agents = [strat_to_bidder(bne_strategies[i], player_position=i, batch_size=eval_batch_size)
              for i in range(n_players)],
    n_players = n_players,
    batch_size = eval_batch_size,
    strategy_to_player_closure = strat_to_bidder
)



#print("Utility in BNE (analytical): \t{:.5f}".format(bne_utility))
bne_utilities_sampled = torch.tensor([bne_env.get_reward(a, draw_valuations = True) for a in bne_env.agents])
print(('Utilities in BNE (sampled):'+ '\t{:.5f}'*n_players + '.').format(*bne_utilities_sampled))

eps_abs = lambda us: bne_utilities_sampled - us
eps_rel = lambda us: 1- us/bne_utilities_sampled

utilities_vs_bne = torch.tensor([bne_env.get_strategy_reward(a.strategy, player_position=i) for i,a in enumerate(env.agents)])
print(('Model utility vs BNE: \t'+'\t{:.5f}'*n_players).format(*utilities_vs_bne))

utilities_learning_env = torch.tensor([env.get_strategy_reward(a.strategy, player_position=i, draw_valuations = True) for i,a in enumerate(env.agents)])
print(('Model utility in learning env:'+'\t{:.5f}'*n_players).format(*utilities_learning_env))

## Training

In [ ]:
if os.name == 'nt': raise ValueError('The run_name may not contain : on Windows! (change datetime format to fix this)') 
run_name = time.strftime('%Y-%m-%d %a %H:%M')
if run_comment:
    run_name = run_name + ' - ' + str(run_comment)
logdir = os.path.join(root_path, 'notebooks', 'asymmetric_fpsb', str(n_players) + 'p', 'uniform', run_name)
print(logdir)


with SummaryWriter(logdir, flush_secs=60) as writer,\
     SummaryWriter(os.path.join(logdir, 'player0')) as writer_1, \
     SummaryWriter(os.path.join(logdir, 'player1')) as writer_2:
    
    player_writers = [writer_1, writer_2]
    
    # create custom_scalar multilinechart with both player's utilities
    ## NOTE: this does not work in pytorch 1.1.0 due to a bug, fixed pytorch-nightly (and will be in 1.2.0 stable)
    # uncomment once 1.2 is available -- see issue #18 https://gitlab.lrz.de/heidekrueger/bnelearn/issues/18
    #writer.add_custom_scalars(torch.utils.tensorb['eval/p1_utility', 'eval/p2_utility'], title = 'Player Utilities')
    
    overhead_mins = 0
    torch.cuda.empty_cache()
    log_once(writer, 0)
    log_hyperparams(writer, 0)
    fig = plt.figure()
    for e in range(epoch+1):
        # lr decay?
        if lr_decay and e % lr_decay_every == 0 and e > 0:
            learning_rate = learning_rate * lr_decay_factor
            log_hyperparams(writer, e)
            for param_group in optimizer_1.param_groups:
                param_group['lr'] = learning_rate

        # always: do optimizer step
        utility_1 = -optimizer_1.step()
        utility_2 = -optimizer_2.step() 
        
        #logging 
        start_time = timer()
        utilities = torch.tensor([utility_1, utility_2])
        utilities_vs_bne = torch.tensor([bne_env.get_strategy_reward(a.strategy, player_position=i) for i,a in enumerate(env.agents)])
        log_utilities_per_player(writer, player_writers, utilities, utilities_vs_bne, e)

        if e % plot_epoch == 0:
            
            # plot current function output
            bidder_1.draw_valuations_()
            v_1 = bidder_1.valuations
            b_1 = bidder_1.get_action()
            bidder_2.draw_valuations_()
            v_2 = bidder_2.valuations
            b_2 = bidder_2.get_action()
            
            #share = b.mean()/optimal_bid(v).mean()
            #diff = (b-optimal_bid(v)).mean()
            #writer.add_scalar('eval/share', share, e)
            #writer.add_scalar('eval/diff', diff, e)
            writer.add_graph(model_1, bidder_1.valuations) 

            print("Epoch {}: \tutilities: \t p1: {:.3f} \t p2: {:.3f}".format(e, utility_1, utility_2))
            plot_bid_function(fig, v_1, b_1, v_2, b_2, writer,e)            
        
        elapsed = timer() - start_time
        overhead_mins = overhead_mins + elapsed/60
        writer.add_scalar('debug/overhead_mins', overhead_mins, e)
            
                     
            